In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [4]:
vocab = sorted(list(set(sentence)))
print(vocab)

['Repeat', 'best', 'for', 'is', 'medicine', 'memory', 'the']


In [5]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [6]:
print(word2index)

{'Repeat': 1, 'best': 2, 'for': 3, 'is': 4, 'medicine': 5, 'memory': 6, 'the': 7, '<unk>': 0}


In [7]:
print(word2index['memory'])

6


In [8]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)


{1: 'Repeat', 2: 'best', 3: 'for', 4: 'is', 5: 'medicine', 6: 'memory', 7: 'the', 0: '<unk>'}


In [9]:
print(index2word[2])

best


In [10]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence]
    input_seq, label_seq = encoded[:-1], encoded[1:]
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치차원 추가 바깥에 []감싸주기
    label_seq = torch.LongTensor(label_seq).unsqueeze(0)
    return input_seq, label_seq


In [11]:
X, Y = build_data(sentence, word2index)

print(X)
print(Y)

tensor([[1, 4, 7, 2, 5, 3]])
tensor([[4, 7, 2, 5, 3, 6]])


In [12]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        output = self.embedding_layer(x)
        output, hidden = self.rnn_layer(output)
        output = self.linear(output)

        return output.view(-1, output.size(2))

In [13]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [14]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [15]:
decode = lambda y: [index2word.get(x) for x in y]

In [16]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.1261 
Repeat Repeat <unk> the <unk> Repeat Repeat

[41/201] 1.5819 
Repeat is the the medicine for memory

[81/201] 0.8830 
Repeat is the best medicine for memory

[121/201] 0.4321 
Repeat is the best medicine for memory

[161/201] 0.2289 
Repeat is the best medicine for memory

[201/201] 0.1347 
Repeat is the best medicine for memory

